In [ ]:
import sqlite3
import geopandas as gpd
import pandas as pd
import geobr
import matplotlib.pyplot as plt
import numpy as np
from shapely.wkt import loads as load_wkt
from shapely.geometry import Point
from PIL import Image
import re

## Defina a pasta de referência

In [ ]:
diretorio = "/home/marchesotti/Grupo A"

# Sincronização com a base de dados

In [ ]:
cnx = sqlite3.connect(f'{diretorio}/indicadores.db')

brazil_df = pd.read_sql_query("SELECT * FROM brazil_df", cnx)
region_df = pd.read_sql_query("SELECT * FROM region_df", cnx)
states_df = pd.read_sql_query("SELECT * FROM states_df", cnx)
cities_df = pd.read_sql_query("SELECT * FROM cities_df", cnx)

# Nivel de Macrorregião

## Coleta dos dados do IBGE

In [ ]:
#Pega os dados geográficos do IBGE
geometry_region = geobr.read_region(year=2018)

In [ ]:
geometry_region

In [ ]:
#Reordena a tabela do ibge na mesma ordem da base de dados
geometry_region = geometry_region.reindex([4,1,0,2,3])
geometry_region

In [ ]:
#Renomeia a coluna de geometry_region para dar o merge com region_df
geometry_region = geometry_region.rename(columns={'name_region':'region'})
geometry_region['region'] = geometry_region['region'].replace(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro Oeste'],['norte', 'nordeste', 'sudeste', 'sul', 'centro-oeste'])
geometry_region

In [ ]:
#Associa o shape a sua respectiva macrorregião
region_df = pd.merge(region_df, geometry_region, how='left', on='region')
region_df

In [ ]:
#Renomear as regiões para escrever corretamente no mapa
geometry_region['region'] = geometry_region['region'].replace(['norte', 'nordeste', 'sudeste', 'sul', 'centro-oeste'],['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro Oeste'])

In [ ]:
#Faz o typecasting do dataframe para GeoDataFrame
geometry_region = gpd.GeoDataFrame(geometry_region)
region_df = gpd.GeoDataFrame(region_df)
#Cria uma nova coluna com as coordenadas do ponto representativo de cada macrorregião
geometry_region['coords'] = geometry_region['geometry'].apply(lambda x: x.representative_point().coords[:])
geometry_region['coords'] = [coords[0] for coords in geometry_region['coords']]

## Letalidade

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(region_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='lethality', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"(Em %)", 'orientation': "vertical"},vmax=region_df['lethality'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Letalidade COVID-19 por Macrorregião (em %)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Macrorregião/Letalidade/semana_{semana}_letalidade_macrorregioes.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Letalidade/semana_{semana}_letalidade_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_letalidade_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Mortalidade

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(region_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a mortalidade
    semana_x.plot(column='mortality', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"(Por 100 mil habitantes)", 'orientation': "vertical"},vmax=region_df['mortality'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Mortalidade COVID-19 por Macrorregião (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Macrorregião/Mortalidade/semana_{semana}_mortalidade_macrorregioes.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Mortalidade/semana_{semana}_mortalidade_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_mortalidade_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Incidência de Casos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(region_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a incidência de casos
    semana_x.plot(column='incidence_cases', cmap='Blues', vmin=0.0, legend=True,legend_kwds={'label': f"(Por 100 mil habitantes)", 'orientation': "vertical"},vmax=region_df['incidence_cases'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Incidência de Casos de COVID-19 por Macrorregião (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Macrorregião/Incidência de Casos/semana_{semana}_incidencia_casos_macrorregioes.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Incidência de Casos/semana_{semana}_incidencia_casos_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_incidencia_casos_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Incidência de Óbitos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(region_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a incidência de óbitos
    semana_x.plot(column='incidence_deaths', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"(Por 100 mil habitantes)", 'orientation': "vertical"},vmax=region_df['incidence_deaths'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Incidência de Óbitos de COVID-19 por Macrorregião (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Macrorregião/Incidência de Óbitos/semana_{semana}_incidencia_obitos_macrorregioes.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Incidência de Óbitos/semana_{semana}_incidencia_obitos_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_incidencia_obitos_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Fator de Crescimento de Casos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(region_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com o fator de crescimento de casos
    semana_x.plot(column='cases_growth_factor', cmap='Blues', vmin=0.0, legend=True,legend_kwds={'label': f"(Em relação a semana epidemiológica anterior)", 'orientation': "vertical"},vmax=region_df['cases_growth_factor'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Fator de Crescimento de Casos de COVID-19 por Macrorregião", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Macrorregião/Fator de Crescimento de Casos/semana_{semana}_crescimento_casos_macrorregioes.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Fator de Crescimento de Casos/semana_{semana}_crescimento_casos_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_crescimento_casos_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Fator de Crescimento de Óbitos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(region_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com o fator de crescimento de óbitos
    semana_x.plot(column='deaths_growth_factor', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"Em relação a semana epidemiológica anterior", 'orientation': "vertical"},vmax=region_df['deaths_growth_factor'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Fator de Crescimento de Óbitos de COVID-19 por Macrorregião", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Macrorregião/Fator de Crescimento de Óbitos/semana_{semana}_crescimento_obitos_macrorregioes.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Fator de Crescimento de Óbitos/semana_{semana}_crescimento_obitos_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_crescimento_obitos_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Prevalência

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(region_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a prevalência
    semana_x.plot(column='prevalence', cmap='Blues', vmin=0.0, legend=True,legend_kwds={'label': f"Por 100 mil habitantes", 'orientation': "vertical"},vmax=region_df['prevalence'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Prevalência de COVID-19 por Macrorregião (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Macrorregião/Prevalência/semana_{semana}_prevalencia_macrorregioes.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Prevalência/semana_{semana}_prevalencia_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_prevalencia_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Casos Acumulados

In [ ]:
#Pré-plota para gerar uma escala para a legenda, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = region_df[region_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.last_available_confirmed/40)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_region['coords']]
ys=[coordenada[1] for coordenada in geometry_region['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#267bb4', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])*40)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2.7, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.last_available_confirmed/20)

    #Plota as linhas dos limites das macrorregiões
    geometry_region.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Casos acumulados por Macrorregião - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Macrorregião/Casos Acumulados/semana_{semana}_casos_acumulados_macrorregioes.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Macrorregião/Casos Acumulados/semana_{semana}_casos_acumulados_macrorregioes.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Macrorregião/Casos Acumulados/semana_{semana}_casos_acumulados_macrorregioes.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Casos Acumulados/semana_{semana}_casos_acumulados_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_casos_acumulados_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Óbitos Acumulados

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de óbitos por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = region_df[region_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.last_available_deaths/1.5)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_region['coords']]
ys=[coordenada[1] for coordenada in geometry_region['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#f24646', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])*1.5)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de óbitos", loc=3,borderpad=3, labelspacing=2.8, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.last_available_deaths/1.5)

    #Plota as linhas dos limites das macrorregiões
    geometry_region.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Óbitos acumulados por Macrorregião - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Macrorregião/Óbitos Acumulados/semana_{semana}_obitos_acumulados_macrorregioes.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Macrorregião/Óbitos Acumulados/semana_{semana}_obitos_acumulados_macrorregioes.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Macrorregião/Óbitos Acumulados/semana_{semana}_obitos_acumulados_macrorregioes.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Óbitos Acumulados/semana_{semana}_obitos_acumulados_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_obitos_acumulados_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Novos Casos

In [ ]:
#Pré-plota para gerar uma escala para a legenda, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = region_df[region_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.new_week_cases/5)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_region['coords']]
ys=[coordenada[1] for coordenada in geometry_region['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#267bb4', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])*5)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2.5, handlelength=7, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.new_week_cases/3)

    #Plota as linhas dos limites das macrorregiões
    geometry_region.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Novos casos por macrorregião - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Macrorregião/Novos Casos/semana_{semana}_novos_casos_macrorregioes.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Macrorregião/Novos Casos/semana_{semana}_novos_casos_macrorregioes.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Macrorregião/Novos Casos/semana_{semana}_novos_casos_macrorregioes.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Novos Casos/semana_{semana}_novos_casos_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_novos_casos_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

## Novos Óbitos

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de óbitos por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = region_df[region_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.new_week_deaths*8)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_region['coords']]
ys=[coordenada[1] for coordenada in geometry_region['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#f24646', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])//8)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de óbitos", loc=3,borderpad=3, labelspacing=2.8, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = region_df[region_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.new_week_deaths*10)

    #Plota as linhas dos limites das macrorregiões
    geometry_region.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_region.iterrows():
        plt.annotate(s=j['region'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Novos óbitos por macrorregião - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Macrorregião/Novos Óbitos/semana_{semana}_novos_obitos_macrorregioes.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Macrorregião/Novos Óbitos/semana_{semana}_novos_obitos_macrorregioes.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Macrorregião/Novos Óbitos/semana_{semana}_novos_obitos_macrorregioes.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Macrorregião/Novos Óbitos/semana_{semana}_novos_obitos_macrorregioes.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_novos_obitos_macrorregioes.gif', save_all=True, append_images=images, duration=500, loop=0)

# Nível de Unidade Federativa

In [ ]:
geometry_states = geobr.read_state(year=2018)

In [ ]:
geometry_states['code_state'] = geometry_states['code_state'].astype('int64') 
geometry_states

In [ ]:
geometry_states = geometry_states.rename(columns={'abbrev_state':'state'})
geometry_states

In [ ]:
states_df = pd.merge(states_df, geometry_states, how='left', on='state')
states_df

In [ ]:
#Cria uma cópia do dataframe para não persistir nenhuma alteração
geometry_states = gpd.GeoDataFrame(geometry_states)
states_df = gpd.GeoDataFrame(states_df)
#Cria uma nova coluna com as coordenadas do ponto representativo de cada macrorregião
geometry_states['coords'] = geometry_states['geometry'].apply(lambda x: x.representative_point().coords[:])
geometry_states['coords'] = [coords[0] for coords in geometry_states['coords']]

## Letalidade

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(states_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='lethality', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"(Em %)", 'orientation': "vertical"},vmax=states_df['lethality'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Letalidade COVID-19 por Unidade Federativa (em %)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Unidade Federativa/Letalidade/semana_{semana}_letalidade_estados.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Letalidade/semana_{semana}_letalidade_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_letalidade_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Mortalidade

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(states_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='mortality', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"(Por 100 mil habitantes)", 'orientation': "vertical"},vmax=states_df['mortality'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Mortalidade COVID-19 por Unidade Federativa (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Unidade Federativa/Mortalidade/semana_{semana}_mortalidade_estados.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Mortalidade/semana_{semana}_mortalidade_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_mortalidade_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Incidência de Casos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(states_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='incidence_cases', cmap='Blues', vmin=0.0, legend=True,legend_kwds={'label': f"(Por 100 mil habitantes)", 'orientation': "vertical"},vmax=states_df['incidence_cases'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Incidência de Casos de COVID-19 por Unidade Federativa (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Unidade Federativa/Incidência de Casos/semana_{semana}_incidencia_casos_estados.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Incidência de Casos/semana_{semana}_incidencia_casos_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_incidencia_casos_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Incidência de Óbitos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(states_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='incidence_deaths', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"(Por 100 mil habitantes)", 'orientation': "vertical"},vmax=states_df['incidence_deaths'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Incidência de Óbitos de COVID-19 por Unidade Federativa (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Unidade Federativa/Incidência de Óbitos/semana_{semana}_incidencia_obitos_estados.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Incidência de Óbitos/semana_{semana}_incidencia_obitos_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_incidencia_obitos_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Fator de Crescimento de Casos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(states_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='cases_growth_factor', cmap='Blues', vmin=0.0, legend=True,legend_kwds={'label': f"(Em relação a semana epidemiológica anterior)", 'orientation': "vertical"},vmax=states_df['cases_growth_factor'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Fator de Crescimento de Casos de COVID-19 por Unidade Federativa", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Unidade Federativa/Fator de Crescimento de Casos/semana_{semana}_crescimento_casos_estados.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Fator de Crescimento de Casos/semana_{semana}_crescimento_casos_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_crescimento_casos_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Fator de Crescimento de Óbitos

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(states_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='deaths_growth_factor', cmap='Reds', vmin=0.0, legend=True,legend_kwds={'label': f"Em relação a semana epidemiológica anterior", 'orientation': "vertical"},vmax=states_df['deaths_growth_factor'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Fator de Crescimento de Óbitos de COVID-19 por Unidade Federativa", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Unidade Federativa/Fator de Crescimento de Óbitos/semana_{semana}_crescimento_obitos_estados.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Fator de Crescimento de Óbitos/semana_{semana}_crescimento_obitos_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_crescimento_obitos_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Prevalência

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica
for semana in range(9,32):
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Cria uma cópia do dataframe para não persistir nenhuma alteração
    df_copy = gpd.GeoDataFrame(states_df)

    #Prepara a base onde as outras camadas do mapa são plotadas
    fig, ax = plt.subplots(figsize=(20, 15))
    #Plota as cores nas macrorregiões de acordo com a letalidade
    semana_x.plot(column='prevalence', cmap='Blues', vmin=0.0, legend=True,legend_kwds={'label': f"Por 100 mil habitantes", 'orientation': "vertical"},vmax=states_df['prevalence'].max(), ax=ax)

    #Escreve qual a semana representada
    ax.annotate(f'Semana {semana}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    #Desenha as linhas de limite entre as macrorregiões
    df_copy.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.1,ax=ax)
    #Escreve o título
    ax.set_title(f"Prevalência de COVID-19 por Unidade Federativa (por 100 mil habitantes)", fontdict={'fontsize':25})
    #Remove os eixos x e y
    ax.axis('off')
    
    #Escreve o nome de cada macrorregião no ponto representativo predefinido
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    #Salva cada mapa como um png
    chart = ax.get_figure()
    chart.savefig(f'{diretorio}/Mapas/Unidade Federativa/Prevalência/semana_{semana}_prevalencia_estados.png', dpi=120)

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Prevalência/semana_{semana}_prevalencia_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_prevalencia_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Casos Acumulados

In [ ]:
#Pré-plota para gerar uma escala para a legenda, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = states_df[states_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.last_available_confirmed/35)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_states['coords']]
ys=[coordenada[1] for coordenada in geometry_states['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#267bb4', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])*35)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2.7, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.last_available_confirmed/20)

    #Plota as linhas dos limites das macrorregiões
    geometry_states.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Casos Acumulados por Unidade Federativa - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Unidade Federativa/Casos Acumulados/semana_{semana}_casos_acumulados_estados.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Unidade Federativa/Casos Acumulados/semana_{semana}_casos_acumulados_estados.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Unidade Federativa/Casos Acumulados/semana_{semana}_casos_acumulados_estados.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Casos Acumulados/semana_{semana}_casos_acumulados_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_casos_acumulados_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Óbitos Acumulados

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de óbitos por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = states_df[states_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.last_available_deaths//1.5)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_states['coords']]
ys=[coordenada[1] for coordenada in geometry_states['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#f24646', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(int(labels[i])*1.5))

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de óbitos", loc=3,borderpad=3, labelspacing=2.8, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.last_available_deaths)

    #Plota as linhas dos limites das macrorregiões
    geometry_states.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Óbitos Acumulados por Unidade Federativa - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Unidade Federativa/Óbitos Acumulados/semana_{semana}_obitos_acumulados_estados.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Unidade Federativa/Óbitos Acumulados/semana_{semana}_obitos_acumulados_estados.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Unidade Federativa/Óbitos Acumulados/semana_{semana}_obitos_acumulados_estados.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Óbitos Acumulados/semana_{semana}_obitos_acumulados_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_obitos_acumulados_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Novos Casos

In [ ]:
#Pré-plota para gerar uma escala para a legenda, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = states_df[states_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.new_week_cases/5)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_states['coords']]
ys=[coordenada[1] for coordenada in geometry_states['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#267bb4', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])*5)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de casos", loc=3,borderpad=3, labelspacing=2.5, handlelength=7, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.new_week_cases/3)

    #Plota as linhas dos limites das macrorregiões
    geometry_states.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Novos Casos por Unidade Federativa - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Unidade Federativa/Novos Casos/semana_{semana}_novos_casos_estados.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Unidade Federativa/Novos Casos/semana_{semana}_novos_casos_estados.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Unidade Federativa/Novos Casos/semana_{semana}_novos_casos_estados.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Novos Casos/semana_{semana}_novos_casos_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_novos_casos_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

## Novos Óbitos

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de óbitos por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = states_df[states_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.new_week_deaths*8)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_states['coords']]
ys=[coordenada[1] for coordenada in geometry_states['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#f24646', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])//8)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de óbitos", loc=3,borderpad=3, labelspacing=2.8, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = states_df[states_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.new_week_deaths*10)

    #Plota as linhas dos limites das macrorregiões
    geometry_states.plot(ax=ax, facecolor='#cccccc')
    semana_x.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Novos Óbitos por Unidade Federativa - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Unidade Federativa/Novos Óbitos/semana_{semana}_novos_obitos_estados.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Unidade Federativa/Novos Óbitos/semana_{semana}_novos_obitos_estados.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Unidade Federativa/Novos Óbitos/semana_{semana}_novos_obitos_estados.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Unidade Federativa/Novos Óbitos/semana_{semana}_novos_obitos_estados.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_novos_obitos_estados.gif', save_all=True, append_images=images, duration=500, loop=0)

# Nível de Município

In [ ]:
geometry_cities = geobr.read_municipality(year=2018)

In [ ]:
geometry_cities['code_muni'] = geometry_cities['code_muni'].astype('int64') 
geometry_cities

In [ ]:
geometry_cities = geometry_cities.rename(columns={'code_muni':'ibge_code'})
geometry_cities

In [ ]:
cities_df['ibge_code'] = cities_df['ibge_code'].astype('int64')

In [ ]:
cities_df = pd.merge(cities_df, geometry_cities, how='left', on='ibge_code')
cities_df

In [ ]:
#Cria uma cópia do dataframe para não persistir nenhuma alteração
geometry_cities = gpd.GeoDataFrame(geometry_cities)
cities_df = gpd.GeoDataFrame(cities_df)
#Cria uma nova coluna com as coordenadas do ponto representativo de cada município
geometry_cities['coords'] = geometry_cities['geometry'].apply(lambda x: x.representative_point().coords[:])
geometry_cities['coords'] = [coords[0] for coords in geometry_cities['coords']]

## Casos Acumulados

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de óbitos por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = cities_df[cities_df.epidemiological_week == 31]

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.last_available_deaths)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_cities['coords']]
ys=[coordenada[1] for coordenada in geometry_cities['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#267bb4', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i]))

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de óbitos", loc=3,borderpad=3, labelspacing=2.8, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = cities_df[cities_df.epidemiological_week == semana]

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.last_available_deaths)

    #Plota as linhas dos limites dos estados
    geometry_states.plot(ax=ax, facecolor='#cccccc')
    geometry_states.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#267bb4")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Óbitos Acumulados por Município - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Município/Casos Acumulados/semana_{semana}_casos_acumulados_municipios.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Município/Casos Acumulados/semana_{semana}_casos_acumulados_municipios.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Município/Casos Acumulados/semana_{semana}_casos_acumulados_municipios.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Município/Casos Acumulados/semana_{semana}_casos_acumulados_municipios.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_casos_acumulados_municipios.gif', save_all=True, append_images=images, duration=500, loop=0)

# Óbitos Acumulados

In [ ]:
#Pré-plot pra gerar uma escala para a legenda de óbitos por municipio, tendo como base a semana 31
fig, ax = plt.subplots(figsize=(20, 20))
semana_x = cities_df[cities_df.epidemiological_week == 31]
semana_x = gpd.GeoDataFrame(semana_x)

markersize = []
for linha in semana_x.itertuples():
    markersize.append(linha.last_available_confirmed/10)
    
semana_x.plot(ax=ax, facecolor='#cccccc')

#Pega as coordenadas que serão os centros dos círculos
xs=[coordenada[0] for coordenada in geometry_cities['coords']]
ys=[coordenada[1] for coordenada in geometry_cities['coords']]
scatter = ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color='#f24646', num=5)
for i in range(len(labels)):
    labels[i] = re.findall(r"\{.+\}", labels[i])[0][1:-1]
    labels[i] = str(int(labels[i])*10)

In [ ]:
#Defineção da legenda
legend ='''ax.legend(handles, labels, title="Número de óbitos", loc=3,borderpad=3, labelspacing=2.8, handlelength=9, title_fontsize=25, fontsize=18, frameon=False)'''
#Loop para cada semana epidemiológica
for semana in range(9, 32):
    #Plota a base do mapa
    fig, ax = plt.subplots(figsize=(20, 20))
    semana_x = cities_df[cities_df.epidemiological_week == semana]
    semana_x = gpd.GeoDataFrame(semana_x)

    #Define o tamanho de cada círculo
    markersize = []
    for linha in semana_x.itertuples():
        markersize.append(linha.last_available_confirmed/10)

    #Plota as linhas dos limites dos estados
    geometry_states.plot(ax=ax, facecolor='#cccccc')
    geometry_states.geometry.boundary.plot(color=None, edgecolor='#444444', linewidth = 0.2, ax=ax)
    
    #Plota os circulos
    ax.scatter(xs, ys, markersize, alpha=0.6, color="#f24646")

    #Desenhar a seta que representa o Norte
    ax.annotate('', xy=(-35, -30), xytext=(-35, -33), arrowprops=dict(arrowstyle= '->', color='k', lw=2, mutation_scale=20))
    ax.text(-35, -30, "N", ha="center", va="bottom", size=20)
    ax.axis('off')

    #Escreve a sigla de cada estado
    for i, j in geometry_states.iterrows():
        plt.annotate(s=j['state'], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)

    #Executa a definição prévia da legenda
    exec(legend)
    #Define o título
    plt.title(f"Óbitos Acumulados por Município - Semana {semana}", fontsize=30)
    #Salva o arquivo da imagem
    plt.savefig(f"{diretorio}/Mapas/Município/Óbitos Acumulados/semana_{semana}_obitos_acumulados_municipios.png")
    #Retira parte das bordas da imagem
    im = Image.open(f"{diretorio}/Mapas/Município/Óbitos Acumulados/semana_{semana}_obitos_acumulados_municipios.png")
    im = im.crop((150,120,1290,1270))
    im.save(f"{diretorio}/Mapas/Município/Óbitos Acumulados/semana_{semana}_obitos_acumulados_municipios.png")

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for semana in range(9,32):
    images.append(Image.open(f'{diretorio}/Mapas/Município/Óbitos Acumulados/semana_{semana}_obitos_acumulados_municipios.png'))

images[0].save(f'{diretorio}/Mapas/Gifs/mapa_obitos_acumulados_municipios.gif', save_all=True, append_images=images, duration=500, loop=0)